<a href="https://colab.research.google.com/github/gustmdtmd/pythondemo/blob/main/python_17_%EC%B6%9C%EC%9E%85%EA%B5%AD_%EA%B4%80%EA%B4%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 구글드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/python_demo

Mounted at /content/drive
/content/drive/MyDrive/python_demo


In [14]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import json
import xmltodict

key = '9YAVHopS3HMY6WSQi9xy6vU7drbecgUOEq+Aj7k4uZRGL+TjtLjukNh9O448Tys15mNLdZgPnZkwixwb4vUwuw=='
#key = '9YAVHopS3HMY6WSQi9xy6vU7drbecgUOEq%2BAj7k4uZRGL%2BTjtLjukNh9O448Tys15mNLdZgPnZkwixwb4vUwuw%3D%3D'

def getTourismStatesItem(yyyymm, national_code, ed_cd):
    url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
    params ={'serviceKey' : key, 'YM' : yyyymm, 'NAT_CD' : national_code, 'ED_CD' : ed_cd }

    response = requests.get(url, params=params)
    #print(response.content)
    #print(response.text)
    return response.text

jsonResult = []
result = []
natName = ''
print('<<국내 입국한 외국인의 통계를 수집합니다.>>')
nat_cd = input('국가 코드를 입력하세요(중국 : 112 / 일본 : 130 / 미국 : 275) :')
nStartYear = int(input('데이터를 몇 년부터 수집할까요? :'))
nEndYear = int(input('데이터를 몇 년까지 수집할까요? :'))
ed_cd = 'E' #E:방한외래관광객, D:해외 출국

dataEND = '{0}{1:0>2}'.format(str(nEndYear), str(12))
print(dataEND)
isDataEnd = 0 # 데이터 끝 확인용 flat 초기화

for year in range(nStartYear, nEndYear+1):
  for month in range(1, 13):
    if isDataEnd == 1: break #데이터 끝 flag 설정되어 있으면 작업 중지
    yyyymm = '{0}{1:0>2}'.format(str(year), str(month))  # >  오른쪽정렬, <  왼쪽정렬, 2 = 자릿수 확보
    
    jsonData = getTourismStatesItem(yyyymm, nat_cd, ed_cd)
    jsonData = xmltodict.parse(jsonData)
    #print(jsonData)

    if jsonData['response']['header']['resultMsg'] == 'OK':
      if jsonData['response']['body']['items'] == '':
        isDataEnd = 1 #데이터 끝 flag 설정
        dataEND = '{0}{1:0>2}'.format(str(year), str(month-1))
        print('데이터 없음...\n 제공되는 통계 데이터는 %s년 %s월까지 입니다.' %str(year), str(month-1))
        break
      
      # jsonData를 출력하여 확인....
      natName = jsonData['response']['body']['items']['item']['natKorNm']
      natName = natName.replace(' ','')
      num = jsonData['response']['body']['items']['item']['num']
      ed = jsonData['response']['body']['items']['item']['ed']
      ym = jsonData['response']['body']['items']['item']['ym']
      print('[%s_%s_%s : %s]' %(ym, natName, ed, num))
  